In [3]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import emcee
import corner
import glob
import math
import re
import scipy
from tqdm import tqdm
from scipy import stats
from scipy.integrate import odeint
from scipy.optimize import minimize
from IPython.display import display, Math
from scipy.optimize import Bounds
import matplotlib.lines as mlines
from multiprocessing import Pool;
from model_functions import log_likelihood_ic, control_tumor

plt.rcParams['font.size'] = '14'

def control_tumor(y, t, theta):
    rval = theta[0] * y[0]*(1-y[0]/K)
    for dtime in ([0,7]):
        if(t-dtime<0):
            return rval
        rval -= y[0]*theta[1]*math.exp(-theta[2]*(t-dtime))
    return rval

def log_likelihood_ic(para):
    theta, times, y, n = para
    ll = 0
    for r in range(n):
        model = odeint(control_tumor, y0=theta[n+r], t=times, args=tuple([[theta[r],theta[r+2*n],theta[3*n]]]))
        variance = theta[3*n+1]**2
        model.shape = y[r,:].shape
        ll += -0.5 * np.sum((y[r,:] - model) ** 2 / variance + np.log(2*np.pi) + np.log(variance))
    return ll

#From Control:
averageIC = 189
rPriorMin = 0.02755937481423266
K = 1169.755238477951

#Load in Theta, times, and y
full_data = []
file_name = '' #Control data location
samples_name = '' #Control flat chain location
scn = 'control'
data_f = np.genfromtxt(file_name, dtype=float, delimiter=',')
data_mice = np.genfromtxt(file_name, dtype=str, delimiter=',')
flat_samples = pd.read_csv(samples_name, delimiter=',',header=None, encoding=None).to_numpy()
full_data.append([data_f,data_mice,flat_samples,scn])
for i in range(1,6):
    file_name = '' + str(i) + '.csv'
    samples_name = '' + str(i) + '.csv' 
    scn = 'Treatment ' + str(i)
    data_f = np.genfromtxt(file_name, dtype=float, delimiter=',')
    data_mice = np.genfromtxt(file_name, dtype=str, delimiter=',')
    flat_samples = pd.read_csv(samples_name, delimiter=',',header=None, encoding=None).to_numpy()
    full_data.append([data_f,data_mice,flat_samples,scn])



if __name__ == "__main__":
    mxlg = [];
    for trtN in range(1,6):
        miceN = int((full_data[trtN][2].shape[1]-2)/3)
        print(miceN)
        flatLen = full_data[trtN][2].shape[0]
        flat = full_data[trtN][2]
        times = [0,7,14]
        yobs = full_data[trtN][0]
        print(flatLen)
        llvalues = [];
        args = [(flat[i], times, yobs[1:,1:-1], miceN) for i in range(flatLen)]
        pbar = tqdm(total=flatLen)
        '''
        for arg in args:
            llvalues.append(log_likelihood_ic(arg));
            pbar.update()
        '''
        with Pool() as pool:
            for result in pool.imap(log_likelihood_ic, args):
                llvalues.append(result)
                pbar.update()
        pbar.close()
        mxlg.append(max(llvalues))
        print("Minimum Log Likelihood Value for Treatment " + str(trtN) + ": " + str(max(llvalues)))

18
1904000


100%|██████████| 1904000/1904000 [55:59<00:00, 566.73it/s]  


Minimum Log Likelihood Value for Treatment 1: -216.9606100710318
10
1088000


100%|██████████| 1088000/1088000 [1:16:42<00:00, 236.37it/s]  


Minimum Log Likelihood Value for Treatment 2: -108.79789275532042
9
986000


...

Minimum Log Likelihood Value for Treatment 3: -100.33139853312917
4
476000


100%|██████████| 476000/476000 [02:08<00:00, 3693.99it/s]


Minimum Log Likelihood Value for Treatment 4: -38.98991240858534
8
884000


100%|██████████| 884000/884000 [08:38<00:00, 1706.21it/s]

Minimum Log Likelihood Value for Treatment 5: -99.51753779426353


In [8]:
mxlg = np.array([-216.9606100710318,-108.79789275532042, -100.33139853312917, -38.98991240858534, -99.51753779426353])
flatLens = np.array([full_data[trtN][2].shape[0] for trtN in range(1,6)])
paramCount = np.array([full_data[trtN][2].shape[1] for trtN in range(1,6)])
BIC = paramCount*np.log(flatLens) - 2*np.array(mxlg)
print(BIC)

[1243.65139982  662.39104012  600.90373444  261.00424868  555.03259647]
